In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import datetime as dt
import datetime
import warnings
import time
import ipywidgets as iw
from IPython.display import display
from datetime import date,timedelta
#from nbsdk import get_table, get_pandas
warnings.filterwarnings('ignore')

plt.rcParams['font.sans-serif'] = ['Microsoft Yahei']  # 用来正常显示中文标签
plt.rcParams['axes.unicode_minus'] = False  # 用来正常显示负号

import seaborn as sns
##登录wind
from WindPy import w
w.start()  # 默认命令超时时间为120秒，如需设置超时时间可以加入waitTime参数，例如waitTime=60,即设置命令超时时间为60秒 
print(w.isconnected())
pd.set_option('display.max_columns', 20)  # 最多展示20列

Welcome to use Wind Quant API for Python (WindPy)!

COPYRIGHT (C) 2020 WIND INFORMATION CO., LTD. ALL RIGHTS RESERVED.
IN NO CIRCUMSTANCE SHALL WIND BE RESPONSIBLE FOR ANY DAMAGES OR LOSSES CAUSED BY USING WIND QUANT API FOR Python.
True


In [46]:
#读取货币周期分割数据
monetary_seperate = pd.read_csv('monetary_seperate.csv',encoding = 'GBK')
monetary_seperate['start_time'] = monetary_seperate['start_time'].astype('datetime64[ns]')
monetary_seperate['end_time'] = monetary_seperate['end_time'].astype('datetime64[ns]')
monetary_seperate = monetary_seperate[['start_time','end_time','type']]

In [47]:
monetary_seperate

,start_time,end_time,type
0,2008-09-01,2009-06-01,宽货币宽信用
1,2009-06-01,2009-12-01,紧货币宽信用
2,2009-12-01,2011-07-01,紧货币紧信用
3,2011-07-01,2012-05-01,宽货币紧信用
4,2012-05-01,2013-05-01,紧货币宽信用
5,2013-05-01,2013-12-01,紧货币紧信用
6,2013-12-01,2015-05-01,宽货币紧信用
7,2015-05-01,2016-09-01,宽货币宽信用
8,2016-09-01,2018-05-01,紧货币紧信用
9,2018-05-01,2018-12-01,宽货币紧信用


In [4]:
#读取hs300牛熊市数据
hs_seperate = pd.read_csv('hs300_seperate.csv')

hs_seperate['start_time'] = pd.to_datetime(hs_seperate['start_time'])
hs_seperate['end_time'] = pd.to_datetime(hs_seperate['end_time'])
hs_seperate = hs_seperate.sort_values('start_time')
#hs_seperate = hs_seperate[hs_seperate['start_time']>=dt.datetime(2018,1,1)]
hs_seperate.reset_index(drop=True,inplace=True)
hs_seperate.columns = ['start_time','end_time','start_index','end_index','type','duration','bench_return']
hs_seperate.tail()

,start_time,end_time,start_index,end_index,type,duration,bench_return
33,2019-01-04,2019-04-22,3036,4026,bull,3.6,32.60%
34,2019-04-22,2020-01-13,4026,4204,others,8.9,4.40%
35,2020-01-13,2020-03-23,4204,3530,bear,2.3,-16.00%
36,2020-03-23,2020-07-13,3530,4853,bull,3.7,37.50%
37,2020-07-13,2020-12-31,4853,5211,others,5.7,7.40%


In [5]:
#测试变量定义
code = '166006.OF'
fields = 'NAV_adj'
beginTime = pd.to_datetime('2008-01-08')
endTime = pd.to_datetime('2020-12-01')
test = w.wsd(code,fields, beginTime, endTime)
hs_field = ['close','pct_chg']
hs_data = w.wsd('000300.SH',hs_field,beginTime,endTime)
df_hs = pd.DataFrame({'time':hs_data.Times,'hs300':hs_data.Data[0],'hs300_return':hs_data.Data[1]})
df_hs.set_index('time',inplace = True)

In [22]:
#牛熊市捏合展示

#account必须为现存未终止的基金
#daydelta必须大于7（最长连续7天非交易日无法提取数据）
#index目前只支持hs300

iw.interact(display_combine_period_return,account_name=iw.Text('166006.OF'),
            start_time = iw.Text('2016-01-08'),
            end_time = iw.Text('2020-01-08'),
            seperate=iw.fixed(hs_seperate))


interactive(children=(Text(value='166006.OF', description='account_name'), Text(value='2016-01-08', descriptio…

<function __main__.display_combine_period_return(account_name, start_time, end_time, seperate, daydelta=20, index='000300.SH')>

In [23]:
#牛熊市分段展示
iw.interact(get_period_return_index,account_name = iw.Text('166006.OF'),
            start_time = iw.Text('2016-01-08'),
            end_time = iw.Text('2020-01-08'),
            seperate=iw.fixed(hs_seperate))

interactive(children=(Text(value='166006.OF', description='account_name'), Text(value='2016-01-08', descriptio…

<function __main__.get_period_return_index(account_name, start_time, end_time, seperate, index='000300.SH', index_name='沪深300')>

In [24]:
display_combine_period_return(code,beginTime,endTime,monetary_seperate)

基金成立日期为：2009-12-30 
输入起始日期必须大于基金成立日期


,#基金代码,#基金名称（中文）,参考指数,基金起始时间,计算区间,时间分割数据起始日期,时间分割数据截止日期
0,166006.OF,中欧行业成长混合(LOF),000300.SH,2009-12-30,2009-12-30 --2020-12-01,2008-09-01,2020-12-01


,市场类型,总持续时长,账户收益,基准收益,超额收益,胜率,分段数
0,紧货币紧信用,1398 days,14.32%,-2.91%,17.23%,0.550725,69
0,宽货币紧信用,1035 days,32.64%,42.55%,-9.90%,0.500000,52
0,紧货币宽信用,365 days,-3.90%,-8.80%,4.90%,0.516129,62
0,宽货币宽信用,976 days,35.93%,-22.28%,58.21%,0.551020,147
0,稳货币宽信用,183 days,40.63%,24.82%,15.81%,0.800000,10
0,稳货币紧信用,61 days,-1.68%,8.24%,-9.93%,0.000000,4


In [25]:
get_period_return_index('166006.OF',pd.to_datetime(beginTime),pd.to_datetime(endTime),monetary_seperate)

实际数据区间为：2009-12-30——2020-12-01 


,起始时间(固定),结束时间(固定),市场类型【沪深300】,持续期,基准收益,账户收益,超额收益,最大回撤,基准最大回撤
2,2009-12-01,2011-07-01,紧货币紧信用,577 days,-14.31%,-3.74%,10.57%,-21.18%,-29.73%
3,2011-07-01,2012-05-01,宽货币紧信用,305 days,-13.89%,-7.01%,6.88%,-25.32%,-27.25%
4,2012-05-01,2013-05-01,紧货币宽信用,365 days,-8.80%,-3.90%,4.90%,-21.61%,-22.41%
5,2013-05-01,2013-12-01,紧货币紧信用,214 days,-0.44%,-9.99%,-9.55%,-21.64%,-18.29%
6,2013-12-01,2015-05-01,宽货币紧信用,516 days,96.37%,73.42%,-22.95%,-17.27%,-15.68%
7,2015-05-01,2016-09-01,宽货币宽信用,489 days,-31.04%,-13.93%,17.11%,-49.85%,-46.70%
8,2016-09-01,2018-05-01,紧货币紧信用,607 days,13.79%,31.94%,18.15%,-10.76%,-14.61%
9,2018-05-01,2018-12-01,宽货币紧信用,214 days,-15.70%,-17.75%,-2.05%,-24.38%,-22.42%
10,2018-12-01,2020-04-01,宽货币宽信用,487 days,12.70%,57.93%,45.23%,-16.59%,-16.08%
11,2020-04-01,2020-10-01,稳货币宽信用,183 days,24.82%,40.63%,15.81%,-9.72%,-7.16%


(1.9552903418999996, 0.42379935468044927, 1.5314909872195503)

In [39]:
#获得基金基本信息

def get_basic_info(code):
    nav = w.wss(code,['name_official','fund_setupdate'])
    name = nav.Data[0][0]
    setup_time = nav.Data[1][0]
    return name,setup_time

In [41]:
name, time = get_basic_info("166006.OF"); time

datetime.datetime(2009, 12, 30, 0, 0)

In [12]:
#拼接净值数据和沪深300指数
def getcompare(code,begintime,endtime,index='000300.SH'):
    nav = w.wsd(code,'NAV_adj', begintime, endtime)
    df_netvalue = pd.DataFrame({'time':nav.Times,'netvalue':nav.Data[0]})
    df_netvalue.set_index('time',inplace = True )
    df_netvalue = df_netvalue.dropna()
    
    index_field = ['close','pct_chg']
    index_data = w.wsd(index,index_field,begintime,endtime)
    
    df_index = pd.DataFrame({'time':index_data.Times,'hs300':index_data.Data[0],'hs300_return':index_data.Data[1]})
    df_index.set_index('time',inplace = True )
    
    compare = pd.concat([df_index,df_netvalue],axis = 1,join='inner')
    return compare

In [43]:
nav = w.wsd(code,'NAV_adj', "2019-05-16", "2020-12-31");

.ErrorCode=0
.Codes=[166006.OF]
.Fields=[NAV_ADJ]
.Times=[20190516,20190517,20190520,20190521,20190522,20190523,20190524,20190527,20190528,20190529,...]
.Data=[[1.5381230123999998,1.5046553664999998,1.477575736,1.5006281393999998,1.4971563919,1.4550788122,1.4563286412999998,1.4749372079,1.4968786521,1.4911849862,...]]

In [31]:
compare = getcompare(code,begintime = "2019-05-16",
           endtime = "2020-12-31",
           index='000300.SH'); compare

,hs300,hs300_return,netvalue
time,,,
2019-05-16,3743.9635,0.452664,1.538123
2019-05-17,3648.7609,-2.542829,1.504655
2019-05-20,3617.7924,-0.848740,1.477576
2019-05-21,3666.7762,1.353969,1.500628
2019-05-22,3649.3796,-0.474439,1.497156
...,...,...,...
2020-12-25,5042.0137,0.839963,3.221643
2020-12-28,5064.4147,0.444287,3.225253
2020-12-29,5042.9361,-0.424108,3.196368


In [14]:
def get_period_return(compare,start_time,end_time):
    '''
    
    计算区间收益

    INPUT：
    compare(df) 计算净值及沪深300全收益，
    start_time（datetime）起始时间，
    end_time（datetime）结束时间，
    
    OUTPUT：
    nv_p_ret(float) 账户收益,
    hs_p_ret(float) 基准收益,
    ex_p_ret(float) 超额收益

    '''
    period_data = compare[start_time:end_time].copy()
    nv_p_ret = period_data.loc[period_data.index[-1],'netvalue']/period_data.loc[period_data.index[0],'netvalue']-1
    hs_p_ret = period_data.loc[period_data.index[-1],'hs300']/period_data.loc[period_data.index[0],'hs300']-1
    ex_p_ret = nv_p_ret-hs_p_ret
    return nv_p_ret,hs_p_ret,ex_p_ret

In [15]:
def get_drawdown(compare,column):
    '''

    INPUT：
    compare(df) 计算净值及沪深300全收益，
    column(str) 计算最大回撤的列
    
    OUTPUT：
    min(dd_tmp_list)(float) 最大回撤
    
    '''
    
    dd_tmp_list = []
    for i in range(len(compare)-1):
        dd_tmp_list.append(compare.loc[compare.index[i+1]:,column].min()/compare.loc[compare.index[i],column]-1)
    return min(dd_tmp_list)

In [16]:
#按照daydelta计算胜率
def get_winprob(compare,seperate,daydelta):
    #计算胜率
    total = 0
    if_win = 0
    begin = seperate.iloc[0]['start_time']
    i=0
    daydelta = timedelta(days=daydelta)
    while i<len(seperate.index):
        #i = i+1
        while begin<=seperate.iloc[i]['end_time']:
            #print('a')
            #begin = end
            end = begin+daydelta
            if(end<=seperate.iloc[i]['end_time']):
                #begin & end 位于同一段市场间
                if len(compare[begin:end])!=0:
                    ex_return = get_period_return(compare,begin,end)[2]
                    if_win = if_win+np.ceil((np.sign(ex_return)+2)/2)-1
                    total = total+1
                begin = end
            else:
                #end在下一段市场内
                #先计算第一期的收益率
                port=1
                bench =1
                if len(compare[begin:seperate.iloc[i]['end_time']])!=0:
                    port,bench,ex_return = get_period_return(compare,begin,seperate.iloc[i]['end_time'])
                    if(i>=len(seperate.index)-1):
                        #无后续牛熊市数据
                        if_win = if_win+np.ceil((np.sign(ex_return)+2)/2)-1
                        total = total+1
                        #print('b')
                    else:
                        #有后续牛熊市
                        daydelta_last = daydelta - (seperate.iloc[i]['end_time']-begin)
                        end = seperate.iloc[i+1]['start_time']+daydelta_last
                        if len(compare[seperate.iloc[i+1]['start_time']:end])!=0:
                            port_add,bench_add,return_add = get_period_return(compare,seperate.iloc[i+1]['start_time'],end)

                            port = (port+1)*(port_add+1)-1
                            bench = (bench+1)*(bench_add+1)-1
                        ex_return = port-bench

                        if_win = if_win+np.ceil((np.sign(ex_return)+2)/2)-1
                        total = total+1
                        #print('c')
                begin = end
        i=i+1
        #print(i)
    
    return if_win/total,total
    
#计算胜率

In [17]:
def get_combine_return(compare,seperate,market_type,daydelta=20):
    #daydelta必须大于7天
    #将牛熊市捏合
    seperate_copy = seperate[seperate['type']==market_type]
    dura = datetime.timedelta(0)
    port_return = 1
    bench_return = 1
    for i in seperate_copy.index:
        if len(compare[seperate_copy.loc[i,'start_time']:seperate_copy.loc[i,'end_time']])!=0:
            
        
            port,bench,ex = get_period_return(compare,seperate_copy.loc[i,'start_time'],seperate_copy.loc[i,'end_time'])
            '''
            port_return = port_return*(get_period_return(compare,seperate_copy.loc[i,'start_time'],seperate_copy.loc[i,'end_time'])[0]+1)
            bench_return = bench_return*(1+get_period_return(compare,seperate_copy.loc[i,'start_time'],seperate_copy.loc[i,'end_time'])[1])
            '''
            port_return = port_return*(port+1)
            bench_return = bench_return*(bench+1)
            #dura = seperate_copy.loc[i,'duration']+dura
            dura = seperate_copy.loc[i,'end_time'] - seperate_copy.loc[i,'start_time'] + dura
    port_return = port_return -1
    bench_return = bench_return -1
    ex_return = port_return - bench_return
    
    winprob,total = get_winprob(compare,seperate_copy,daydelta)
    return market_type,dura,port_return, bench_return,ex_return,winprob,total

In [50]:
get_combine_return(compare, monetary_seperate, "紧货币紧信用")

ZeroDivisionError: division by zero

In [18]:
def display_combine_period_return(account_name,start_time,end_time,seperate,daydelta=20,index = '000300.SH'):
    
    start_time = pd.to_datetime(start_time)
    end_time = pd.to_datetime(end_time)
    name,setup_time = get_basic_info(account_name)
    if start_time<setup_time:
        print('基金成立日期为：'+str(setup_time)[:11])
        print('输入起始日期必须大于基金成立日期')
        start_time = setup_time
    
    type_list = seperate[(seperate['end_time']>start_time)&(seperate['start_time']<end_time)]['type'].unique()
    #type_list = seperate['type'].unique()
    
    #time_gap = datetime.timedelta(days =time_gap)
    #end_time = seperate.iloc[-1]['end_time']
    #end_time = pd.to_datetime('2019/10/10')
    start_time = max(start_time,seperate.iloc[0]['start_time'])
    
    basicinfo = pd.DataFrame({
        '#基金代码':[account_name],
        '#基金名称（中文）':[name],
        '参考指数':[index],
        '基金起始时间':[str(setup_time)[:11]],
        '计算区间':[str(start_time)[:11]+'--'+str(end_time)[:11]],
        '时间分割数据起始日期':[seperate.iloc[0]['start_time']],
        '时间分割数据截止日期':[seperate.iloc[-1]['end_time']]
                    
                }
                )
    
    display(basicinfo)
    compare = getcompare(account_name,start_time,end_time,index)
    #print(compare.head(5))
    #print(compare)
    result = pd.DataFrame()
    for i in type_list:
        #print(i)
        result = result.append(pd.DataFrame(get_combine_return(compare,seperate,i,daydelta)).T)
    #print(result)
    #result = result.append(pd.DataFrame(get_combine_return(compare,seperate,'bear',daydelta)).T)
    #result = result.append(pd.DataFrame(get_combine_return(compare,seperate,'others',daydelta)).T)
    
    
    result.columns = ['市场类型','总持续时长','账户收益','基准收益','超额收益','胜率','分段数']
    result['账户收益'] = result['账户收益'].map(lambda x:str('%.2f' %(x*100))+'%' if x!='-' else '-')
    result['基准收益'] = result['基准收益'].map(lambda x:str('%.2f' %(x*100))+'%' if x!='-' else '-')
    result['超额收益'] = result['超额收益'].map(lambda x:str('%.2f' %(x*100))+'%' if x!='-' else '-')
    result = result.replace({'bull':'牛市','bear':'熊市','others':'震荡市'})
    
    return result

In [19]:
def display_allindex_period_return(account_name,start_time,end_time,hs_seperate,sz_seperate,cy_seperate,an_rate=0.0914):
    compare = getcompare(account_name,start_time,end_time)
    
    
    nv_p_ret,hs_p_ret,hs_ex_p_ret = get_period_return_index(account_name,start_time,end_time,hs_seperate,'000300.SH','沪深300',an_rate)
    nv_p_ret,sz_p_ret,sz_ex_p_ret = get_period_return_index(account_name,start_time,end_time,sz_seperate,'000001.SH','上证指数',an_rate)
    nv_p_ret,cy_p_ret,cy_ex_p_ret = get_period_return_index(account_name,start_time,end_time,cy_seperate,'399006.SZ','创业板指',an_rate)
    
    print(start_time)
    print('账户起始:',str(compare.index[0])[:10])
    time.sleep(0.3)
    try:
        if start_time < compare.index[0]:
            print('时间超出限制，起始至少应大于账户成立日',str(compare.index[0])[:10])
            period_ret = pd.DataFrame(columns=['#账户名称','#账户名称（中文）','#时间区间','区间收益','区间基准收益','区间超额收益'])
        else:
            nv_p_ret,hs_p_ret,ex_p_ret = get_period_return(compare,start_time,end_time)
            period_ret = pd.DataFrame({
                    '#账户名称':[account_name],
                    '#账户名称（中文）':[account_name],
                    '时间区间':[str(start_time)[:11]+' - '+str(end_time)[:11]],
                    '区间收益':[str(format((nv_p_ret*100),'.2f'))+'%'],
                    '沪深基准收益':[str(format((hs_p_ret*100),'.2f'))+'%'],
                    '沪深超额收益':[str(format((hs_ex_p_ret*100),'.2f'))+'%'],
                    '上证基准收益':[str(format((sz_p_ret*100),'.2f'))+'%'],
                    '上证超额收益':[str(format((sz_ex_p_ret*100),'.2f'))+'%'],
                    '创业基准收益':[str(format((cy_p_ret*100),'.2f'))+'%'],
                    '创业超额收益':[str(format((cy_ex_p_ret*100),'.2f'))+'%']
                }
                )

    except IndexError:
        print('时间超出限制，起始至少应小于结束日期')#,str(compare.index[0])[:10])
        period_ret = pd.DataFrame({
                '#账户名称':['-'],
                '#账户名称（中文）':['-'],
                '#时间区间':['-'],
                '区间收益':['-'],
                '沪深基准收益':['-'],
                '沪深超额收益':['-'],
                '上证基准收益':['-'],
                '上证超额收益':['-'],
                '创业基准收益':['-'],
                '创业超额收益':['-']
            }
            )
    return period_ret

In [20]:
def get_period_return_index(account_name,start_time,end_time,seperate,index='000300.SH',index_name='沪深300'):
    #account_en_name = gp_name.loc[gp_name['vc_gp3_account_name']==account_name,'vc_gp3_account'].values[0]
    '''
    if start_time> end_time:
        end_time = seperate['end_time'][-1]
    '''
    start_time = pd.to_datetime(start_time)
    end_time = pd.to_datetime(end_time)
    compare = getcompare(account_name,start_time,end_time,index)
    #print(compare.head(5))
    #print(start_time)
    #ex_p_rate = '-'
    start_time = max(start_time,compare.index[0])
    print('实际数据区间为：'+str(start_time)[:11]+'——'+str(end_time)[:11])
    nv_p_ret,hs_p_ret,ex_p_ret = get_period_return(compare,start_time,end_time)
    '''
    if start_time >= compare.index[0]:
        nv_p_ret,hs_p_ret,ex_p_ret = get_period_return(compare,start_time,end_time)
    else:
        
        nv_p_ret,hs_p_ret,ex_p_ret = get_period_return(compare,compare.index[0],end_time)
    '''
    seperate_copy = seperate.copy()[['start_time','end_time','type']]
    seperate_copy['dura']= seperate_copy['end_time']-seperate_copy['start_time']
    seperate_copy['port_return'] = 0
    seperate_copy['excess_return'] = 0
    
    for i in range(len(seperate_copy)):
        if seperate_copy.loc[i,'end_time']<compare.index[0]:
            
            seperate_copy.loc[i,'port_return'] = '-'
            seperate_copy.loc[i,'bench_return'] = '-'
            seperate_copy.loc[i,'excess_return'] = '-'  
            seperate_copy.loc[i,'drawdown'] = '-'  
            seperate_copy.loc[i,'bench_drawdown'] = '-'  
            
        else:
            if seperate_copy.loc[i,'start_time']>compare.index[-1]:
                seperate_copy.loc[i,'port_return'] = '-'
                seperate_copy.loc[i,'bench_return'] = '-'
                seperate_copy.loc[i,'excess_return'] = '-'
                seperate_copy.loc[i,'drawdown'] = '-'  
                seperate_copy.loc[i,'bench_drawdown'] = '-'
            else:
                seperate_copy.loc[i,'port_return'] = round(100*get_period_return(compare,seperate_copy.loc[i,'start_time'],seperate_copy.loc[i,'end_time'])[0],2)
                seperate_copy.loc[i,'bench_return'] = round(100*get_period_return(compare,seperate_copy.loc[i,'start_time'],seperate_copy.loc[i,'end_time'])[1],2)
                seperate_copy.loc[i,'excess_return'] = (round(seperate_copy.loc[i,'port_return']-float(seperate_copy.loc[i,'bench_return']),2))
                seperate_copy.loc[i,'drawdown'] = (round(100*get_drawdown(compare[seperate_copy.loc[i,'start_time']:seperate_copy.loc[i,'end_time']],'netvalue'),2))
                seperate_copy.loc[i,'bench_drawdown'] = (round(100*get_drawdown(compare[seperate_copy.loc[i,'start_time']:seperate_copy.loc[i,'end_time']],'hs300'),2))

    
    

    #display(seperate)
    seperate_copy['port_return'] = seperate_copy['port_return'].map(lambda x:str('%.2f' %x)+'%' if x!='-' else '-')
    seperate_copy['excess_return'] = seperate_copy['excess_return'].map(lambda x:str('%.2f' %x)+'%' if x!='-' else '-')
    seperate_copy['bench_return'] = seperate_copy['bench_return'].map(lambda x:str('%.2f' %x)+'%' if x!='-' else '-')
    seperate_copy['drawdown'] = seperate_copy['drawdown'].map(lambda x:str('%.2f' %x)+'%' if x!='-' else '-')
    seperate_copy['bench_drawdown'] = seperate_copy['bench_drawdown'].map(lambda x:str('%.2f' %x)+'%' if x!='-' else '-')

    '''
    seperate_copy.columns = ['起始时间(固定)','结束时间(固定)','起始指数','结束指数','市场类型【'+index_name+'】','持续期/月','基准收益','账户收益','超额收益','最大回撤','基准最大回撤']
    seperate_copy = seperate_copy[['起始时间(固定)','结束时间(固定)','市场类型【'+index_name+'】','持续期/月','基准收益','账户收益','超额收益','最大回撤','基准最大回撤']]
    seperate_copy = seperate_copy.replace({'bull':'牛市','bear':'熊市','others':'震荡市'})
    '''
    #print(seperate_copy.columns)
    seperate_copy.columns = ['起始时间(固定)','结束时间(固定)','市场类型【'+index_name+'】','持续期','账户收益','超额收益','基准收益','最大回撤','基准最大回撤']
    seperate_copy = seperate_copy[['起始时间(固定)','结束时间(固定)','市场类型【'+index_name+'】','持续期','基准收益','账户收益','超额收益','最大回撤','基准最大回撤']]
    #seperate_copy = seperate_copy.replace({'bull':'牛市','bear':'熊市','others':'震荡市'})
    display(seperate_copy[seperate_copy['基准收益']!='-'])
    
    return nv_p_ret,hs_p_ret,ex_p_ret

In [21]:
get_period_return_index('166006.OF',beginTime,endTime,hs_seperate)

实际数据区间为：2009-12-30——2020-12-01 


,起始时间(固定),结束时间(固定),市场类型【沪深300】,持续期,基准收益,账户收益,超额收益,最大回撤,基准最大回撤
15,2009-11-24,2010-07-02,bear,220 days,-28.79%,-18.84%,9.95%,-20.69%,-29.35%
16,2010-07-02,2010-11-11,bull,132 days,38.51%,39.45%,0.94%,-5.02%,-4.47%
17,2010-11-11,2011-01-25,bear,75 days,-16.28%,-15.92%,0.36%,-15.92%,-16.28%
18,2011-01-25,2011-04-18,bull,83 days,14.32%,8.70%,-5.62%,-3.70%,-4.25%
19,2011-04-18,2012-01-06,bear,263 days,-31.82%,-27.55%,4.27%,-27.55%,-32.24%
20,2012-01-06,2012-05-04,bull,119 days,18.57%,21.78%,3.21%,-8.65%,-8.88%
21,2012-05-04,2012-12-04,bear,214 days,-21.52%,-8.48%,13.04%,-21.61%,-22.41%
22,2012-12-04,2013-02-08,bull,66 days,30.04%,18.81%,-11.23%,-5.52%,-2.08%
23,2013-02-08,2013-06-25,bear,137 days,-21.87%,-20.59%,1.28%,-21.64%,-21.87%
24,2013-06-25,2013-10-14,others,111 days,14.18%,9.51%,-4.67%,-12.38%,-6.48%


(1.9552903418999996, 0.42379935468044927, 1.5314909872195503)

In [ ]:
display_combine_period_return(code,beginTime,endTime,hs_seperate)

In [ ]:
get_period_return_index('000011.OF',pd.to_datetime('2005-04-08'),pd.to_datetime(endTime),hs_seperate)